In [1]:
#Laod current hamiltonian file
using LinearAlgebra, Plots, Graphs, ProgressBars, Base.Threads, Statistics
include("../src/graphs.jl")
include("../src/observables.jl")

hamiltonian_from_graph (generic function with 4 methods)

In [2]:
# parameters
SIZE_LIST = [50, 100, 200] # number of sites
DISORDER_LIST = 10 .^ LinRange(-2, 2, 26) 
NUM_SAMPLES = 10


10

In [3]:
function compute_ipr_per_size_and_disorder(sizes, disorders, num_samples)
    # get localization of the ground state as a function of disorder
    iprs = zeros(length(sizes), length(disorders))
    iprs_std = zeros(length(sizes), length(disorders))
    for (size_idx, L) in enumerate(sizes)
        graph = chain_graph(L)
    
        for (disorder_idx, disorder) in enumerate(disorders)

            ipr_sample_arr = zeros(num_samples)
            Threads.@threads for n in 1:num_samples
                chain_hamiltonian = hamiltonian_from_graph(graph, disorder * rand(nv(graph)), 1.0) 
                eigval_vec, eigvec_mat  = eigen(chain_hamiltonian)
                ipr_sample_arr[n] = inverse_participation_ratio(eigvec_mat[:, 1])
            end
            iprs[size_idx, disorder_idx] = mean(ipr_sample_arr)
            iprs_std[size_idx, disorder_idx] = stdm(ipr_sample_arr, iprs[size_idx, disorder_idx])
        end
    end
    return iprs, iprs_std
end

iprs_arr, iprs_std_arr = compute_ipr_per_size_and_disorder(SIZE_LIST, DISORDER_LIST, NUM_SAMPLES)


TaskFailedException: TaskFailedException

    nested task error: UndefVarError: inverse_participation_ratio not defined
    Stacktrace:
     [1] macro expansion
       @ ~/Dropbox/progra/julia/PKS_Julia_Hackathon/notebooks/other_grahps.ipynb:14 [inlined]
     [2] (::var"#66#threadsfor_fun#9"{SimpleGraph{Int64}, Float64, Vector{Float64}, UnitRange{Int64}})(onethread::Bool)
       @ Main ./threadingconstructs.jl:85
     [3] (::var"#66#threadsfor_fun#9"{SimpleGraph{Int64}, Float64, Vector{Float64}, UnitRange{Int64}})()
       @ Main ./threadingconstructs.jl:52

In [4]:
# Look at iprs as function of disorder
plot_inst = scatter(
        # xaxis = :log,
        scale = :log,
        xlabel = "Disorder Strength",
        ylabel = "Ground State IPR",
        legend = :topleft,
        legendtitle = "L"
)

for (L, ipr, ipr_std) in zip(SIZE_LIST, eachrow(iprs_arr), eachrow(iprs_std_arr))
    scatter!(DISORDER_LIST, ipr, yerror=ipr_std, label = L)
end

plot_inst

UndefVarError: UndefVarError: iprs_arr not defined